# Import libraries

In [198]:
import pickle
import numpy as np
import pandas as pd

# Loading and exploring data

In [199]:
df_cases = pd.read_pickle("../data/df_cases_200906.gzip")
df_label = pd.read_pickle("../data/df_label_200906.gzip")

In [200]:
df_cases.head(5)

,CaseId,FileName,Language,StartDate,DocumentType,IsExecuted,OcrText,QualityScore
0,003061226227,003061226227_12045631_Order form_978-1-67767-3...,EN,20191104,Order form,False,Pressure style response character.,0.649292
1,003061226383,003061226383_63912371_Contract Info Pack_978-0...,EN,20190805,Contract Info Pack,False,Soon especially boy thousand traditional.,0.873038
2,003061226383,003061226383_41775424_other documents_978-1-38...,EN,20190805,other documents,True,Tax south say strategy hard between late.,0.991280
3,003061227777,003061227777_96463321_other documents_978-1-96...,EN,20191231,other documents,False,Yet arm six design.,0.607833
4,003061227781,003061227781_62201704_other documents_978-1-04...,EN,20191231,other documents,False,Plan camera actually run push rest.,0.909359


In [201]:
df_label.head(5)

,CaseId,label_1,label_2
0,003061226227,False,False
1,003061226383,True,False
2,003061227777,False,False
3,003061227781,False,False
4,003061227680,True,True


# Cleaning data

In [202]:
### 1. Filtering invalid columns ###
original_row_count = df_cases.values.shape[0]

# define filtering criteria
criteria = np.where((df_cases['IsExecuted']==True) & (df_cases['QualityScore'] >= 0.81)) # for valid
criteria_inv = np.where((df_cases['IsExecuted']==False) | (df_cases['QualityScore'] < 0.81)) # for invalid

df_cases_invalid = df_cases.iloc[criteria_inv[0], :]
df_cases = df_cases.iloc[criteria[0], :]

new_row_count = df_cases.values.shape[0]
num_unique_rows = len(np.unique(df_cases['CaseId'].values))

print(f'[INFO] {original_row_count - new_row_count} rows filtered')
print(f'[INFO] Number of rows remaining : {new_row_count}')
print(f'[INFO] Number of unique rows remaining : {num_unique_rows}')

print('[INFO] List of filtered rows : ')
df_cases_invalid[['CaseId', 'IsExecuted', 'QualityScore']]

[INFO] 1389 rows filtered
[INFO] Number of rows remaining : 680
[INFO] Number of unique rows remaining : 570
[INFO] List of filtered rows : 


,CaseId,IsExecuted,QualityScore
0,003061226227,False,0.649292
1,003061226383,False,0.873038
3,003061227777,False,0.607833
4,003061227781,False,0.909359
5,003061227680,False,0.829900
...,...,...,...
2327,003061208632,True,0.730278
2328,003061208849,True,0.693007
2330,003061207200,True,0.745751
2331,003061209028,True,0.713114


In [203]:
### 2. Get only the necessary columns ###
df_cases = df_cases[['CaseId', 'FileName', 'OcrText']]

# Concatenate OcrText column

In [204]:
### 3. Concatenate OcrText columns using aggregation ###
def concat_by(delimiter): 
    # define aggregation method
    concat = lambda x : delimiter.join(x)
    return concat

df_cases = df_cases.groupby('CaseId').agg({                               
    'FileName' : concat_by(','), # Concatenate file name by colon
    'OcrText':concat_by(' ')     # Concatenate ocr text by space
})

df_cases

,FileName,OcrText
CaseId,,
003061189006,003061189006_69176036_Order form_978-1-62414-9...,As difficult behavior her myself help.
003061189067,003061189067_26173467_Order form_978-1-904782-...,Present can phone form.
003061189156,003061189156_47966765_Contract Documents_978-0...,Section science difference success wish it wide.
003061189229,003061189229_21094545_other documents_978-1-81...,Accept sell leader herself if.
003061189349,003061189349_72196625_Other documents_978-0-60...,Another later everybody large real.
...,...,...
003061230596,003061230596_91328991_other documents_978-0-88...,Miss style unit. High begin purpose interestin...
003061230613,003061230613_55291460_other documents_978-1-17...,Really certainly might responsibility responsi...
003061230657,003061230657_50860326_Order form_978-1-80159-5...,Cultural light carry past technology finish la...


# Separate filename into InvalidFileNames and ValidFileNames

In [205]:
### 4. Separate FileName into InvalidFileNames and ValidFileNames ###
'''
    Since the documentation does not specify the criterias of a valid file name. I will assume
    that a valid filename does not consist white space.
'''
# This function will be applied in FileName column to find valid file names
def get_filename(row, is_valid=True):
    filenames = row.split(',')
    num_files = len(filenames) # number of files in this row
    
    result = '['    
    for i, name in enumerate(filenames):    
        if(is_valid):
            if(' ' in name):
                continue
            else:
                result += name
        else:
            if(' ' not in name):
                continue
            else:
                result += name
                
        if(i != num_files - 1):
            result += ", "
            
    result += "]"
    return result

df_cases['ValidFileNames'] = df_cases['FileName'].apply(lambda row : get_filename(row))
df_cases['InvalidFileNames'] = df_cases['FileName'].apply(lambda row : get_filename(row, is_valid=False))

# Merging df_cases to df_label

In [206]:
### Finally, merge df_cases and df_label on 'CaseId' ###
result = df_cases.merge(df_label, on='CaseId')
result = result[['CaseId', 'InvalidFileNames', 'ValidFileNames', 'OcrText', 'label_1', 'label_2']]

print('[INFO] Saving final DataFrame to pickle ... ')
result.to_pickle('df_final.gzip')

print('[INFO] Sample records of final DataFrame : ')
result.tail(5)

[INFO] Saving final DataFrame to pickle ... 
[INFO] Sample records of final DataFrame : 


,CaseId,InvalidFileNames,ValidFileNames,OcrText,label_1,label_2
565,003061230596,[003061230596_91328991_other documents_978-0-8...,[],Miss style unit. High begin purpose interestin...,False,False
566,003061230613,[003061230613_55291460_other documents_978-1-1...,[],Really certainly might responsibility responsi...,False,False
567,003061230657,[003061230657_50860326_Order form_978-1-80159-...,[],Cultural light carry past technology finish la...,True,False
568,003061230728,[],[003061230728_74076581_Amendments_978-0-14-763...,None attorney spend tend miss appear.,True,False
569,003061230757,[003061230757_72990476_Contract Documents_978-...,[],Determine go network.,False,False
